In [15]:
import petl as etl
import pandas as pd

In [16]:
# lecture du fichier CSV
table = etl.fromcsv("test.csv")

# affichage du resultat
etl.look(table)

+-----+-----------+-----------------+------------+
| id  | name      | age             | city       |
+=====+===========+=================+============+
| '1' | 'Alice'   | '25'            | 'New York' |
+-----+-----------+-----------------+------------+
| '2' | 'Bob'     | 'not_available' | 'London'   |
+-----+-----------+-----------------+------------+
| '3' | 'Charlie' | '30'            | 'Sydney'   |
+-----+-----------+-----------------+------------+
| '4' | 'David'   | '22'            | 'New York' |
+-----+-----------+-----------------+------------+

### toutes les data sont du type string en sortie de lecture

#### la fonction .convert va nous permettre d'appliquer a chaque valeur de la colonne age en etype entier

In [17]:
table = (
    table
    .convert('age', lambda v: int(v) if v else None)
)
etl.look(table)

+-----+-----------+------+------------+
| id  | name      | age  | city       |
+=====+===========+======+============+
| '1' | 'Alice'   |   25 | 'New York' |
+-----+-----------+------+------------+
| '2' | 'Bob'     | None | 'London'   |
+-----+-----------+------+------------+
| '3' | 'Charlie' |   30 | 'Sydney'   |
+-----+-----------+------+------------+
| '4' | 'David'   |   22 | 'New York' |
+-----+-----------+------+------------+

#### .select va nous permettre de filtrer le tableau selon une ou plusieurs conditions

#### la condition est l'age compris en tre 20 et 30 ans 

In [18]:
table2 = (
    table   
    .select(lambda row: row.age is not None and row.age > 20 and row.age < 30)
)
etl.look(table2)

+-----+---------+-----+------------+
| id  | name    | age | city       |
+=====+=========+=====+============+
| '1' | 'Alice' |  25 | 'New York' |
+-----+---------+-----+------------+
| '4' | 'David' |  22 | 'New York' |
+-----+---------+-----+------------+

#### .cut va restreindre le tableqau a certaines colonnes

In [19]:
table3 = (
    table2
    .cut('name', 'city')
)
etl.look(table3)

+---------+------------+
| name    | city       |
+=========+============+
| 'Alice' | 'New York' |
+---------+------------+
| 'David' | 'New York' |
+---------+------------+

#### continuons avec un fichier de 500 lignes account_list.csv

In [20]:
data = etl.fromcsv("account_list.csv")
etl.look(data)

+-------------------+---------------------------------------------------------+------------------------------+------------------------+------------------------------------+------------------+---------------+-----------------------+----------------------+--------------------+--------------------------+
| name              | address                                                 | email                        | phone_number           | job                                | company          | date_of_birth | credit_card_number    | credit_card_provider | credit_card_expire | iban                     |
+===================+=========================================================+==============================+========================+====================================+==================+===============+=======================+======================+====================+==========================+
| 'Ian Yang'        | '5143 Garcia Shoal Suite 003\nJustintown, CA 11547'     | 'jessicanob

In [21]:
# nettoyer la colonne 'address' en supprimant les retours chariot et en normalisant les espaces
data_clean = data.convert('address', lambda v: ' '.join(v.split()) if v else v)

# affichage du résultat
etl.look(data_clean)

+-------------------+--------------------------------------------------------+------------------------------+------------------------+------------------------------------+------------------+---------------+-----------------------+----------------------+--------------------+--------------------------+
| name              | address                                                | email                        | phone_number           | job                                | company          | date_of_birth | credit_card_number    | credit_card_provider | credit_card_expire | iban                     |
+===================+========================================================+==============================+========================+====================================+==================+===============+=======================+======================+====================+==========================+
| 'Ian Yang'        | '5143 Garcia Shoal Suite 003 Justintown, CA 11547'     | 'jessicanoble@e

#### creation de colonne first_name et last_name puis sppression de la colonne name

In [22]:
data_clean = (
    data_clean
    .addfield('first_name', lambda row: row.name.split()[0].strip("'"))
    .addfield('last_name', lambda row: row.name.split()[1].strip("'"))
    .cutout('name')
)
etl.look(data_clean)

+--------------------------------------------------------+------------------------------+------------------------+------------------------------------+------------------+---------------+-----------------------+----------------------+--------------------+--------------------------+------------+-------------+
| address                                                | email                        | phone_number           | job                                | company          | date_of_birth | credit_card_number    | credit_card_provider | credit_card_expire | iban                     | first_name | last_name   |
+========================================================+==============================+========================+====================================+==================+===============+=======================+======================+====================+==========================+============+=============+
| '5143 Garcia Shoal Suite 003 Justintown, CA 11547'     | 'jessicanoble@

In [23]:
data_clean = (
    data_clean
    .addfield('phone', lambda row: row.phone_number.split('x')[0].strip() if 'x' in row.phone_number else row.phone_number)
    .addfield('bureau', lambda row: row.phone_number.split('x')[1].strip() if 'x' in row.phone_number else None)
    .cutout('phone_number')
)
etl.look(data_clean)

+--------------------------------------------------------+------------------------------+------------------------------------+------------------+---------------+-----------------------+----------------------+--------------------+--------------------------+------------+-------------+-------------------+---------+
| address                                                | email                        | job                                | company          | date_of_birth | credit_card_number    | credit_card_provider | credit_card_expire | iban                     | first_name | last_name   | phone             | bureau  |
+========================================================+==============================+====================================+==================+===============+=======================+======================+====================+==========================+============+=============+===================+=========+
| '5143 Garcia Shoal Suite 003 Justintown, CA 11547'     |

In [24]:
# Nettoie et standardise les valeurs de credit_card_expire
data_clean = (
    data_clean
    # Convertie en datetime et reformate en "DD-MM-YY" credit_card_expire date
    .convert('credit_card_expire', lambda v: pd.to_datetime(f"01/{v}", format="%d/%m/%y").strftime("%d-%m-%y"))
)
etl.look(data_clean)

+--------------------------------------------------------+------------------------------+------------------------------------+------------------+---------------+-----------------------+----------------------+--------------------+--------------------------+------------+-------------+-------------------+---------+
| address                                                | email                        | job                                | company          | date_of_birth | credit_card_number    | credit_card_provider | credit_card_expire | iban                     | first_name | last_name   | phone             | bureau  |
+========================================================+==============================+====================================+==================+===============+=======================+======================+====================+==========================+============+=============+===================+=========+
| '5143 Garcia Shoal Suite 003 Justintown, CA 11547'     |

#### combinaison de plusieurs actions sur le tableau

In [25]:
data2 = (
    data
    .addfield('first_name', lambda row: row.name.split()[0].strip("'"))
    .addfield('last_name', lambda row: row.name.split()[1].strip("'"))
    .cutout('name')
    .addfield('phone', lambda row: row.phone_number.split('x')[0].strip() if 'x' in row.phone_number else row.phone_number)
    .addfield('bureau', lambda row: row.phone_number.split('x')[1].strip() if 'x' in row.phone_number else None)
    .cutout('phone_number')
    .convert('credit_card_expire', lambda v: pd.to_datetime(f"01/{v}", format="%d/%m/%y").strftime("%d-%m-%y"))
    .select(lambda row: row.credit_card_provider == 'VISA 16 digit' 
           and pd.to_datetime(row.credit_card_expire, format='%d-%m-%y') >= pd.to_datetime('01-01-25', format='%d-%m-%y'))
)
etl.look(data2)

+---------------------------------------------------------+--------------------------------+-----------------------------------------------+------------------+---------------+-----------------------+----------------------+--------------------+--------------------------+------------+-------------+-------------------+---------+
| address                                                 | email                          | job                                           | company          | date_of_birth | credit_card_number    | credit_card_provider | credit_card_expire | iban                     | first_name | last_name   | phone             | bureau  |
+=========================================================+================================+===============================================+==================+===============+=======================+======================+====================+==========================+============+=============+===================+=========+
| '9873 John For

### loading dans un nouveau fichier csv

In [27]:
etl.tocsv(data2, "clean_and_filter.csv")